---
## 1. Import Required Libraries

**Interview Explanation:**
- **Pandas**: Data manipulation and analysis
- **NumPy**: Numerical operations
- **Matplotlib & Seaborn**: Data visualization
- **Datetime**: Time-series data handling

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time and date handling
from datetime import datetime, timedelta
import warnings

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All libraries imported successfully!")

---
## 2. Load the Dataset

**Interview Explanation:**
- Loading the raw energy dataset
- Displaying first few rows to understand structure
- Checking basic information about columns and data types

In [ ]:
# Load the dataset
df_raw = pd.read_csv('energy_dataset.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df_raw.shape[0]} rows, {df_raw.shape[1]} columns")
print(f"\nFirst 5 rows:")
df_raw.head()

In [ ]:
# Display basic information
print("Dataset Information:")
print("="*50)
df_raw.info()

In [ ]:
# Display column names for better understanding
print("\nColumn Names:")
print("="*50)
for i, col in enumerate(df_raw.columns, 1):
    print(f"{i}. {col}")

---
## 3. Data Cleaning & Preprocessing

**Interview Explanation:**
This is a critical step where we:
1. **Convert time column** to proper datetime format for time-series analysis
2. **Handle missing values** - Check percentage and decide strategy
3. **Remove duplicates** - Ensure data quality
4. **Select relevant columns** - Focus on consumption and generation data
5. **Create new features** - Extract hour, day, month for trend analysis

In [ ]:
# Step 1: Create a working copy
df = df_raw.copy()

print("Step 1: Working copy created")
print(f"Original dataset preserved with {df_raw.shape[0]} rows")

In [ ]:
# Step 2: Convert time column to datetime
df['time'] = pd.to_datetime(df['time'], utc=True)

# Set time as index for time-series analysis
df.set_index('time', inplace=True)

print("Step 2: Time column converted to datetime")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Total duration: {(df.index.max() - df.index.min()).days} days")

In [ ]:
# Step 3: Check for missing values
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Percentage': missing_percent
}).sort_values('Missing_Count', ascending=False)

print("Step 3: Missing Values Analysis")
print("="*60)
print(missing_df[missing_df['Missing_Count'] > 0])

total_missing = df.isnull().sum().sum()
print(f"\nTotal missing values: {total_missing}")

In [ ]:
# Step 4: Check for duplicates
duplicates = df.duplicated().sum()
print(f"Step 4: Found {duplicates} duplicate rows")

if duplicates > 0:
    df.drop_duplicates(inplace=True)
    print(f"Removed {duplicates} duplicates")

In [ ]:
# Step 5: Select relevant columns
# Removing forecast columns and price columns for this analysis
# Focus on actual generation and consumption

columns_to_drop = [
    'forecast solar day ahead',
    'forecast wind offshore eday ahead', 
    'forecast wind onshore day ahead',
    'total load forecast',
    'price day ahead',
    'price actual',
    'generation hydro pumped storage aggregated',
    'generation fossil coal-derived gas',  # Has many missing values
    'generation fossil oil shale',  # Has many missing values
    'generation fossil peat',  # Has many missing values
    'generation geothermal',  # Has many missing values
    'generation marine'  # Has many missing values
]

df_clean = df.drop(columns=columns_to_drop, errors='ignore')

print("Step 5: Removed irrelevant and high-missing columns")
print(f"Columns reduced from {df.shape[1]} to {df_clean.shape[1]}")

In [ ]:
# Step 6: Handle remaining missing values using forward fill
# For time-series data, forward fill is appropriate as energy patterns are continuous

before_fill = df_clean.isnull().sum().sum()
df_clean = df_clean.fillna(method='ffill').fillna(method='bfill')
after_fill = df_clean.isnull().sum().sum()

print("Step 6: Missing values handled")
print(f"Missing values before: {before_fill}")
print(f"Missing values after: {after_fill}")
print("\n✓ Data cleaning completed!")

In [ ]:
# Step 7: Feature Engineering - Extract time-based features
# These features help identify patterns in energy consumption

df_clean['Year'] = df_clean.index.year
df_clean['Month'] = df_clean.index.month
df_clean['Day'] = df_clean.index.day
df_clean['Hour'] = df_clean.index.hour
df_clean['DayOfWeek'] = df_clean.index.dayofweek  # Monday=0, Sunday=6
df_clean['DayName'] = df_clean.index.day_name()
df_clean['MonthName'] = df_clean.index.month_name()
df_clean['Quarter'] = df_clean.index.quarter

# Classify time of day
def classify_time(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df_clean['TimeOfDay'] = df_clean['Hour'].apply(classify_time)

# Weekend flag
df_clean['IsWeekend'] = df_clean['DayOfWeek'].isin([5, 6]).astype(int)

print("Step 7: Time-based features created")
print("\nNew features:")
print("- Year, Month, Day, Hour")
print("- DayOfWeek, DayName, MonthName")
print("- Quarter, TimeOfDay, IsWeekend")
print(f"\nFinal dataset shape: {df_clean.shape}")

In [ ]:
# Display cleaned dataset summary
print("\n" + "="*70)
print("CLEANED DATASET SUMMARY")
print("="*70)
df_clean.info()

---
## 4. Exploratory Data Analysis (EDA)

**Interview Explanation:**
EDA helps us understand:
- **Distribution** of energy consumption
- **Statistical summary** of all variables
- **Correlations** between different energy sources
- **Trends** over time
- **Patterns** in consumption behavior

In [ ]:
# Statistical Summary
print("Statistical Summary of Energy Data:")
print("="*70)
df_clean.describe().round(2)

In [ ]:
# 1. Overall Energy Consumption Distribution
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df_clean['total load actual'], bins=50, color='steelblue', edgecolor='black')
plt.xlabel('Energy Load (MW)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Total Energy Load', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(df_clean['total load actual'], vert=True)
plt.ylabel('Energy Load (MW)', fontsize=12)
plt.title('Boxplot of Total Energy Load', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean Load: {df_clean['total load actual'].mean():.2f} MW")
print(f"Median Load: {df_clean['total load actual'].median():.2f} MW")
print(f"Max Load: {df_clean['total load actual'].max():.2f} MW")
print(f"Min Load: {df_clean['total load actual'].min():.2f} MW")

In [ ]:
# 2. Energy Generation by Source
generation_columns = [
    'generation biomass',
    'generation fossil brown coal/lignite',
    'generation fossil gas',
    'generation fossil hard coal',
    'generation fossil oil',
    'generation hydro pumped storage consumption',
    'generation hydro run-of-river and poundage',
    'generation hydro water reservoir',
    'generation nuclear',
    'generation other',
    'generation other renewable',
    'generation solar',
    'generation waste',
    'generation wind offshore',
    'generation wind onshore'
]

# Calculate total generation for each source
generation_totals = df_clean[generation_columns].sum().sort_values(ascending=False)

plt.figure(figsize=(14, 8))
colors = plt.cm.Spectral(np.linspace(0, 1, len(generation_totals)))
bars = plt.bar(range(len(generation_totals)), generation_totals.values, color=colors)
plt.xticks(range(len(generation_totals)), 
           [col.replace('generation ', '') for col in generation_totals.index], 
           rotation=45, ha='right')
plt.xlabel('Energy Source', fontsize=12)
plt.ylabel('Total Generation (MW)', fontsize=12)
plt.title('Total Energy Generation by Source', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nTop 5 Energy Sources:")
for i, (source, value) in enumerate(generation_totals.head().items(), 1):
    print(f"{i}. {source.replace('generation ', '').title()}: {value:,.0f} MW")

In [ ]:
# 3. Renewable vs Non-Renewable Energy
renewable_sources = [
    'generation biomass',
    'generation hydro run-of-river and poundage',
    'generation hydro water reservoir',
    'generation other renewable',
    'generation solar',
    'generation wind offshore',
    'generation wind onshore'
]

non_renewable_sources = [
    'generation fossil brown coal/lignite',
    'generation fossil gas',
    'generation fossil hard coal',
    'generation fossil oil',
    'generation nuclear'
]

df_clean['Total_Renewable'] = df_clean[renewable_sources].sum(axis=1)
df_clean['Total_NonRenewable'] = df_clean[non_renewable_sources].sum(axis=1)

renewable_total = df_clean['Total_Renewable'].sum()
non_renewable_total = df_clean['Total_NonRenewable'].sum()

# Pie chart
plt.figure(figsize=(10, 6))
sizes = [renewable_total, non_renewable_total]
labels = ['Renewable', 'Non-Renewable']
colors = ['#2ecc71', '#e74c3c']
explode = (0.05, 0)

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 14, 'fontweight': 'bold'})
plt.title('Renewable vs Non-Renewable Energy Generation', fontsize=16, fontweight='bold')
plt.axis('equal')
plt.show()

print(f"\nRenewable Energy: {renewable_total:,.0f} MW ({renewable_total/(renewable_total+non_renewable_total)*100:.1f}%)")
print(f"Non-Renewable Energy: {non_renewable_total:,.0f} MW ({non_renewable_total/(renewable_total+non_renewable_total)*100:.1f}%)")

In [ ]:
# 4. Time Series Plot - Overall Consumption Trend
plt.figure(figsize=(16, 6))
plt.plot(df_clean.index, df_clean['total load actual'], linewidth=0.5, color='darkblue', alpha=0.7)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Load (MW)', fontsize=12)
plt.title('Energy Consumption Over Time (2015-2018)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Key Observations:")
print("- Seasonal patterns are visible")
print("- Regular cyclical variations indicate daily/weekly patterns")
print("- No significant missing data gaps")

---
## 5. Peak Consumption Analysis

**Interview Explanation:**
Identifying peak hours helps:
- **Grid Management**: Plan capacity during high-demand periods
- **Cost Optimization**: Implement dynamic pricing
- **Resource Allocation**: Schedule maintenance during low-demand hours
- **Business Strategy**: Target energy-saving campaigns

In [ ]:
# 1. Average Load by Hour of Day
hourly_avg = df_clean.groupby('Hour')['total load actual'].mean()

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
bars = plt.bar(hourly_avg.index, hourly_avg.values, color='coral', edgecolor='black')
# Highlight peak hour
peak_hour = hourly_avg.idxmax()
bars[peak_hour].set_color('red')
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Average Load (MW)', fontsize=12)
plt.title('Average Energy Consumption by Hour', fontsize=14, fontweight='bold')
plt.xticks(range(24))
plt.grid(True, alpha=0.3, axis='y')

# Line plot for better trend visualization
plt.subplot(1, 2, 2)
plt.plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2, markersize=8, color='darkgreen')
plt.axvline(x=peak_hour, color='red', linestyle='--', linewidth=2, label=f'Peak Hour: {peak_hour}:00')
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Average Load (MW)', fontsize=12)
plt.title('Hourly Consumption Trend', fontsize=14, fontweight='bold')
plt.xticks(range(24))
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 PEAK HOUR ANALYSIS")
print("="*50)
print(f"Peak Hour: {peak_hour}:00 ({hourly_avg[peak_hour]:.2f} MW)")
print(f"Lowest Hour: {hourly_avg.idxmin()}:00 ({hourly_avg.min():.2f} MW)")
print(f"Difference: {hourly_avg[peak_hour] - hourly_avg.min():.2f} MW")

# Identify peak hours (above 95th percentile)
threshold = hourly_avg.quantile(0.85)
peak_hours = hourly_avg[hourly_avg >= threshold].index.tolist()
print(f"\nPeak Hours (Top 15%): {peak_hours}")

In [ ]:
# 2. Average Load by Day of Week
daily_avg = df_clean.groupby('DayName')['total load actual'].mean()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_avg = daily_avg.reindex(day_order)

plt.figure(figsize=(12, 6))
colors_day = ['green' if day in ['Saturday', 'Sunday'] else 'steelblue' for day in day_order]
plt.bar(range(7), daily_avg.values, color=colors_day, edgecolor='black')
plt.xticks(range(7), day_order, rotation=45)
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Average Load (MW)', fontsize=12)
plt.title('Average Energy Consumption by Day of Week', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.legend(['Weekday', 'Weekend'], loc='upper right')
plt.tight_layout()
plt.show()

print("\n📅 DAILY PATTERN ANALYSIS")
print("="*50)
for day, load in daily_avg.items():
    print(f"{day}: {load:.2f} MW")

In [ ]:
# 3. Weekday vs Weekend Comparison
weekday_avg = df_clean[df_clean['IsWeekend'] == 0]['total load actual'].mean()
weekend_avg = df_clean[df_clean['IsWeekend'] == 1]['total load actual'].mean()

plt.figure(figsize=(10, 6))
categories = ['Weekday', 'Weekend']
values = [weekday_avg, weekend_avg]
colors = ['#3498db', '#2ecc71']
bars = plt.bar(categories, values, color=colors, edgecolor='black', width=0.6)

# Add value labels on bars
for bar, value in zip(bars, values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{value:.0f} MW',
             ha='center', va='bottom', fontsize=14, fontweight='bold')

plt.ylabel('Average Load (MW)', fontsize=12)
plt.title('Weekday vs Weekend Energy Consumption', fontsize=14, fontweight='bold')
plt.ylim(0, max(values) * 1.1)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

difference = weekday_avg - weekend_avg
percent_diff = (difference / weekday_avg) * 100
print(f"\n📈 WEEKDAY VS WEEKEND COMPARISON")
print("="*50)
print(f"Weekday Average: {weekday_avg:.2f} MW")
print(f"Weekend Average: {weekend_avg:.2f} MW")
print(f"Difference: {difference:.2f} MW ({percent_diff:.2f}% lower on weekends)")

In [ ]:
# 4. Monthly Consumption Pattern
monthly_avg = df_clean.groupby('MonthName')['total load actual'].mean()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_avg = monthly_avg.reindex(month_order)

plt.figure(figsize=(14, 6))
colors_month = plt.cm.coolwarm(np.linspace(0, 1, 12))
plt.bar(range(12), monthly_avg.values, color=colors_month, edgecolor='black')
plt.xticks(range(12), [m[:3] for m in month_order], rotation=0)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Load (MW)', fontsize=12)
plt.title('Average Energy Consumption by Month', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n📆 MONTHLY PATTERN ANALYSIS")
print("="*50)
print(f"Highest Consumption: {monthly_avg.idxmax()} ({monthly_avg.max():.2f} MW)")
print(f"Lowest Consumption: {monthly_avg.idxmin()} ({monthly_avg.min():.2f} MW)")
print("\nObservation: Higher consumption in winter months (heating) and summer months (cooling)")

In [ ]:
# 5. Heatmap: Hour vs Day of Week
heatmap_data = df_clean.pivot_table(
    values='total load actual',
    index='Hour',
    columns='DayName',
    aggfunc='mean'
)
heatmap_data = heatmap_data[day_order]

plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, annot=False, fmt='.0f', cmap='YlOrRd', cbar_kws={'label': 'Load (MW)'})
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Hour of Day', fontsize=12)
plt.title('Energy Consumption Heatmap: Hour vs Day', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔥 HEATMAP INSIGHTS")
print("="*50)
print("- Darker colors indicate higher consumption")
print("- Clear pattern of peak hours (9-20) visible")
print("- Weekend consumption lower than weekdays")
print("- Night hours (0-6) show lowest consumption")

---
## 6. Business Insights & KPIs

**Interview Explanation:**
These KPIs are crucial for business decision-making:
- **Total Load**: Overall energy demand
- **Peak Load**: Maximum capacity required
- **Average Load**: Baseline operations
- **Load Factor**: Efficiency metric (Average/Peak ratio)
- **Renewable Percentage**: Sustainability metric

In [ ]:
# Calculate Key Performance Indicators (KPIs)
total_load = df_clean['total load actual'].sum()
peak_load = df_clean['total load actual'].max()
average_load = df_clean['total load actual'].mean()
min_load = df_clean['total load actual'].min()
load_factor = (average_load / peak_load) * 100

renewable_percentage = (renewable_total / (renewable_total + non_renewable_total)) * 100

# Growth analysis
yearly_avg = df_clean.groupby('Year')['total load actual'].mean()
yearly_growth = yearly_avg.pct_change() * 100

print("\n" + "="*70)
print("KEY PERFORMANCE INDICATORS (KPIs)")
print("="*70)
print(f"\n📊 LOAD METRICS:")
print(f"   Total Load: {total_load:,.0f} MW")
print(f"   Peak Load: {peak_load:,.0f} MW")
print(f"   Average Load: {average_load:,.0f} MW")
print(f"   Minimum Load: {min_load:,.0f} MW")
print(f"   Load Factor: {load_factor:.2f}%")

print(f"\n🌱 SUSTAINABILITY METRICS:")
print(f"   Renewable Energy: {renewable_percentage:.2f}%")
print(f"   Non-Renewable Energy: {100-renewable_percentage:.2f}%")

print(f"\n📈 YEAR-OVER-YEAR GROWTH:")
for year in yearly_avg.index:
    print(f"   {year}: {yearly_avg[year]:,.0f} MW", end="")
    if year in yearly_growth.index and not pd.isna(yearly_growth[year]):
        print(f" ({yearly_growth[year]:+.2f}% vs previous year)")
    else:
        print()

print(f"\n⚡ PEAK INSIGHTS:")
print(f"   Peak Hour: {peak_hour}:00")
print(f"   Peak Day: {daily_avg.idxmax()}")
print(f"   Peak Month: {monthly_avg.idxmax()}")

print("\n" + "="*70)

In [ ]:
# Business Recommendations Dashboard
print("\n" + "="*70)
print("BUSINESS RECOMMENDATIONS")
print("="*70)

recommendations = [
    "1. PEAK DEMAND MANAGEMENT:\n"
    f"   - Peak consumption occurs at {peak_hour}:00\n"
    "   - Implement demand response programs during peak hours\n"
    "   - Consider dynamic pricing to shift load to off-peak hours\n",
    
    "2. RENEWABLE ENERGY OPTIMIZATION:\n"
    f"   - Current renewable energy: {renewable_percentage:.1f}%\n"
    f"   - Target: Increase to {renewable_percentage + 10:.1f}% in next year\n"
    "   - Invest in solar/wind capacity during low-generation periods\n",
    
    "3. OPERATIONAL EFFICIENCY:\n"
    f"   - Load Factor: {load_factor:.1f}% indicates room for improvement\n"
    "   - Schedule maintenance during low-demand hours (2 AM - 5 AM)\n"
    "   - Optimize grid operations during weekend (lower demand)\n",
    
    "4. SEASONAL PLANNING:\n"
    f"   - Highest demand: {monthly_avg.idxmax()}\n"
    "   - Prepare additional capacity for summer/winter peaks\n"
    "   - Promote energy conservation campaigns in high-demand months\n",
    
    "5. COST OPTIMIZATION:\n"
    "   - Weekday demand is higher than weekend\n"
    "   - Implement time-of-use pricing strategies\n"
    "   - Encourage industrial users to shift operations to off-peak times\n"
]

for rec in recommendations:
    print(rec)

---
## 7. Export Data for Power BI Dashboard

**Interview Explanation:**
We export multiple CSV files for Power BI:
1. **Complete cleaned dataset** - Full historical data
2. **Daily aggregates** - For daily trend analysis
3. **Hourly aggregates** - For intraday patterns
4. **Monthly aggregates** - For long-term trends
5. **KPI summary** - For dashboard cards

In [ ]:
# 1. Export complete cleaned dataset
df_clean_export = df_clean.copy()
df_clean_export.reset_index(inplace=True)
df_clean_export.to_csv('cleaned_energy_data.csv', index=False)
print("✓ Exported: cleaned_energy_data.csv")
print(f"  Rows: {len(df_clean_export)}, Columns: {len(df_clean_export.columns)}")

In [ ]:
# 2. Export daily aggregates
daily_summary = df_clean.groupby(df_clean.index.date).agg({
    'total load actual': ['sum', 'mean', 'max', 'min'],
    'Total_Renewable': 'sum',
    'Total_NonRenewable': 'sum',
    'generation solar': 'sum',
    'generation wind onshore': 'sum',
    'generation wind offshore': 'sum'
}).reset_index()

daily_summary.columns = ['Date', 'Total_Load', 'Avg_Load', 'Peak_Load', 'Min_Load',
                         'Renewable_Gen', 'NonRenewable_Gen', 'Solar_Gen', 
                         'Wind_Onshore_Gen', 'Wind_Offshore_Gen']

daily_summary.to_csv('daily_energy_summary.csv', index=False)
print("✓ Exported: daily_energy_summary.csv")
print(f"  Rows: {len(daily_summary)}")

In [ ]:
# 3. Export hourly pattern data
hourly_pattern = df_clean.groupby(['Hour', 'DayName']).agg({
    'total load actual': 'mean',
    'Total_Renewable': 'mean',
    'Total_NonRenewable': 'mean'
}).reset_index()

hourly_pattern.columns = ['Hour', 'Day', 'Avg_Load', 'Avg_Renewable', 'Avg_NonRenewable']
hourly_pattern.to_csv('hourly_pattern.csv', index=False)
print("✓ Exported: hourly_pattern.csv")
print(f"  Rows: {len(hourly_pattern)}")

In [ ]:
# 4. Export monthly summary
monthly_summary = df_clean.groupby(['Year', 'MonthName']).agg({
    'total load actual': ['sum', 'mean', 'max'],
    'Total_Renewable': 'sum',
    'Total_NonRenewable': 'sum'
}).reset_index()

monthly_summary.columns = ['Year', 'Month', 'Total_Load', 'Avg_Load', 'Peak_Load',
                           'Renewable_Gen', 'NonRenewable_Gen']

monthly_summary.to_csv('monthly_energy_summary.csv', index=False)
print("✓ Exported: monthly_energy_summary.csv")
print(f"  Rows: {len(monthly_summary)}")

In [ ]:
# 5. Export KPI summary for Power BI cards
kpi_data = pd.DataFrame({
    'Metric': ['Total Load (MW)', 'Peak Load (MW)', 'Average Load (MW)', 
               'Load Factor (%)', 'Renewable Energy (%)', 'Peak Hour',
               'Peak Day', 'Peak Month'],
    'Value': [f"{total_load:,.0f}", f"{peak_load:,.0f}", f"{average_load:,.0f}",
              f"{load_factor:.2f}", f"{renewable_percentage:.2f}", 
              f"{peak_hour}:00", daily_avg.idxmax(), monthly_avg.idxmax()]
})

kpi_data.to_csv('kpi_summary.csv', index=False)
print("✓ Exported: kpi_summary.csv")
print(f"  Rows: {len(kpi_data)}")

In [ ]:
# Summary of all exported files
print("\n" + "="*70)
print("EXPORTED FILES SUMMARY")
print("="*70)
print("\nFiles ready for Power BI Dashboard:")
print("1. cleaned_energy_data.csv - Complete dataset with all features")
print("2. daily_energy_summary.csv - Daily aggregates for trend analysis")
print("3. hourly_pattern.csv - Hourly patterns by day of week")
print("4. monthly_energy_summary.csv - Monthly summaries for YoY comparison")
print("5. kpi_summary.csv - Key metrics for dashboard cards")
print("\n✓ All files exported successfully!")
print("\nNext Step: Import these CSVs into Power BI to create visualizations")

---
## 8. Project Summary & Key Findings

### Key Achievements:
1. ✓ Cleaned and preprocessed 35,000+ energy consumption records
2. ✓ Handled missing values using forward-fill imputation
3. ✓ Engineered 10+ time-based features for analysis
4. ✓ Identified peak consumption patterns and trends
5. ✓ Generated actionable business insights
6. ✓ Prepared data exports for Power BI dashboard

### Technical Skills Demonstrated:
- **Python**: Pandas, NumPy, Matplotlib, Seaborn
- **Data Cleaning**: Missing value handling, duplicate removal, data type conversion
- **Feature Engineering**: Time-based features, aggregations, categorical encoding
- **EDA**: Statistical analysis, visualization, pattern identification
- **Business Intelligence**: KPI calculation, insights generation, data export

### Next Steps:
1. Create Power BI dashboard with interactive visualizations
2. Implement predictive models for load forecasting
3. Deploy as web application for stakeholder access